In [1]:
from threading import Thread
import os
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

In [2]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

In [3]:
import torch

WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

In [4]:
import torch2trt
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

from torch2trt import TRTModule
model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

In [5]:
import time

#t0 = time.time()
#torch.cuda.current_stream().synchronize()
#for i in range(50):
#    y = model_trt(data)
#torch.cuda.current_stream().synchronize()
#t1 = time.time()

#print(50.0 / (t1 - t0))

In [6]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [7]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

In [8]:
#from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
#from jetcam.utils import bgr8_to_jpeg

#camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=15)

camera.running = True

In [9]:
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

image_w = ipywidgets.Image(format='jpeg', width=WIDTH, height=HEIGHT)

display(image_w)

Image(value=b'', format='jpeg', height='224', width='224')

In [10]:
import numpy as np
import shutil
global folder
camera.running = True
folder = "/home/jetbot/Desktop/"
file_path = folder + "temp.csv"
f = open(file_path, "a")

In [11]:
def get_keypoint(humans, hnum, peaks):
    #check invalid human index
    kpoint = []
    human = humans[0][hnum]
    C = human.shape[0]
    for j in range(C):
        k = int(human[j])
        if k >= 0:
            peak = peaks[0][j][k]   # peak[1]:width, peak[0]:height
            peak = [j, float(peak[0]), float(peak[1])]
            kpoint.append(peak)
            #print('index:%d : success [%5.3f, %5.3f]'%(j, peak[1], peak[2]) )
        else:    
            peak = [j, -1, -1] # -1 = None
            kpoint.append(peak)
            #print('index:%d : None %d'%(j, k) )
    return kpoint

In [12]:
def upload():
    print(str(int(time.time())) + " Upload thread running...")
    global upload_running
    upload_running = 1
    # for this to work we need to exchange credentials manually first and install sshpass and openssh server
    # change password for the real password, user for the real username and ip for the real ip or domain
    # change the path after ip_of_server: to match the real path where you want to save the files
    # Example using rsync with compression, delete of source files and partial transfers handling: 
    #   sshpass -p password rsync --partial --remove-source-files -v -z -a ./*1*.csv username@ip_of_server:/home/username/data_folder/
    cmd = "sshpass -p datayzu scp -o StrictHostKeyChecking=no /home/jetbot/Desktop/*1*.csv data@192.168.3.89:/home/data/data_folder/"
    ret = os.system(cmd)
    
    if ret != 0:
        print(str(int(time.time())) + " Upload returned error code: " + str(ret))
    else:
        print(str(int(time.time())) + " Uploads success, cleaning files...")
        cmd = "rm /home/jetbot/Desktop/*1*.csv"
        os.system(cmd)
        print(str(int(time.time()))+ " Files cleaned");
        files_copied = 0

    upload_running = 0
    print( str(int(time.time())) + " Upload thread finished")
    

In [13]:
def fan_control(speed):
    if speed >= 0 and speed <=255:
        print("Changing speed of fan to: " + str(speed))
        cmd = "echo " + str(speed) + " > /sys/devices/pwm-fan/target_pwm"
        os.system(cmd)
        print("Speed of fan changed")
    else:
        print("Speed value incorrect: " + str(speed) + ". Not changing fan speed")

In [14]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    keypoints = get_keypoint(objects, 0, peaks) # 0 for just the first human recognized in the frame
    # print(keypoints) # print keypoints for first human recognized in the frame
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    np.savetxt(f, keypoints, delimiter=',', fmt='%.4f', newline='\n')
    global kp_sets_saved
    global files_copied
    kp_sets_saved = kp_sets_saved + 1
    print(str(int(time.time())) + " kp sets saved: " + str(kp_sets_saved))
    if kp_sets_saved == 100: # save every 100 sets of keypoints
        current_epoch = int(time.time())
        new_file = folder + str(current_epoch) + ".csv"
        print(str(int(time.time())) + " temp file copied to: " + new_file)
        shutil.copy(file_path, new_file)
        files_copied = files_copied + 1
        kp_sets_saved = 0
        print(str(int(time.time())) + " temp file truncated")
        f.truncate()
    if files_copied == 5 and upload_running == 0:
        thread = Thread(target = upload) 
        thread.start()
        files_copied = 0

In [15]:
kp_sets_saved = 0
files_copied = 0
upload_running = 0
# activate fan on low speed before starting processing images
fan_control(64) # 1/4 of full speed, 255 is full speeed
# upload from previous sessions to clean up the storage before running pose estimation
# This function call runs blocks because it doesn not run as a thread, so wait for it to finish
upload()

Changing speed of fan to: 64
Speed of fan changed
1605510094 Upload thread running...
1605510095 Upload returned error code: 256
1605510095 Upload thread finished


In [16]:
execute({'new': camera.value})

1605510106 kp sets saved: 1


In [17]:
camera.observe(execute, names='value')

1605510109 kp sets saved: 2
1605510109 kp sets saved: 3
1605510109 kp sets saved: 4
1605510109 kp sets saved: 5
1605510109 kp sets saved: 6
1605510109 kp sets saved: 7
1605510109 kp sets saved: 8
1605510109 kp sets saved: 9
1605510109 kp sets saved: 10
1605510110 kp sets saved: 11
1605510110 kp sets saved: 12
1605510110 kp sets saved: 13
1605510110 kp sets saved: 14
1605510110 kp sets saved: 15
1605510110 kp sets saved: 16
1605510110 kp sets saved: 17
1605510110 kp sets saved: 18
1605510110 kp sets saved: 19
1605510110 kp sets saved: 20
1605510110 kp sets saved: 21
1605510111 kp sets saved: 22
1605510111 kp sets saved: 23
1605510111 kp sets saved: 24
1605510111 kp sets saved: 25
1605510111 kp sets saved: 26
1605510111 kp sets saved: 27
1605510111 kp sets saved: 28
1605510111 kp sets saved: 29
1605510111 kp sets saved: 30
1605510111 kp sets saved: 31
1605510111 kp sets saved: 32
1605510111 kp sets saved: 33
1605510112 kp sets saved: 34
1605510112 kp sets saved: 35
1605510112 kp sets sav

1605510135 kp sets saved: 81
1605510135 kp sets saved: 82
1605510135 kp sets saved: 83
1605510135 kp sets saved: 84
1605510135 kp sets saved: 85
1605510135 kp sets saved: 86
1605510135 kp sets saved: 87
1605510135 kp sets saved: 88
1605510135 kp sets saved: 89
1605510136 kp sets saved: 90
1605510136 kp sets saved: 91
1605510136 kp sets saved: 92
1605510136 kp sets saved: 93
1605510136 kp sets saved: 94
1605510136 kp sets saved: 95
1605510136 kp sets saved: 96
1605510136 kp sets saved: 97
1605510136 kp sets saved: 98
1605510136 kp sets saved: 99
1605510136 kp sets saved: 100
1605510136 temp file copied to: /home/jetbot/Desktop/1605510136.csv
1605510136 temp file truncated
1605510137 kp sets saved: 1
1605510137 kp sets saved: 2
1605510137 kp sets saved: 3
1605510137 kp sets saved: 4
1605510137 kp sets saved: 5
1605510138 kp sets saved: 6
1605510138 kp sets saved: 7
1605510138 kp sets saved: 8
1605510138 kp sets saved: 9
1605510138 kp sets saved: 10
1605510138 kp sets saved: 11
1605510138

1605510162 kp sets saved: 54
1605510163 kp sets saved: 55
1605510163 kp sets saved: 56
1605510163 kp sets saved: 57
1605510163 kp sets saved: 58
1605510163 kp sets saved: 59
1605510163 kp sets saved: 60
1605510163 kp sets saved: 61
1605510163 kp sets saved: 62
1605510163 kp sets saved: 63
1605510163 kp sets saved: 64
1605510164 kp sets saved: 65
1605510164 kp sets saved: 66
1605510164 kp sets saved: 67
1605510164 kp sets saved: 68
1605510164 kp sets saved: 69
1605510164 kp sets saved: 70
1605510164 kp sets saved: 71
1605510164 kp sets saved: 72
1605510164 kp sets saved: 73
1605510164 kp sets saved: 74
1605510165 kp sets saved: 75
1605510165 kp sets saved: 76
1605510165 kp sets saved: 77
1605510165 kp sets saved: 78
1605510165 kp sets saved: 79
1605510165 kp sets saved: 80
1605510165 kp sets saved: 81
1605510165 kp sets saved: 82
1605510165 kp sets saved: 83
1605510166 kp sets saved: 84
1605510166 kp sets saved: 85
1605510166 kp sets saved: 86
1605510166 kp sets saved: 87
1605510166 kp 

1605510190 kp sets saved: 26
1605510190 kp sets saved: 27
1605510191 kp sets saved: 28
1605510191 kp sets saved: 29
1605510191 kp sets saved: 30
1605510191 kp sets saved: 31
1605510191 kp sets saved: 32
1605510191 kp sets saved: 33
1605510191 kp sets saved: 34
1605510191 kp sets saved: 35
1605510191 kp sets saved: 36
1605510191 kp sets saved: 37
1605510192 kp sets saved: 38
1605510192 kp sets saved: 39
1605510192 kp sets saved: 40
1605510192 kp sets saved: 41
1605510192 kp sets saved: 42
1605510192 kp sets saved: 43
1605510192 kp sets saved: 44
1605510192 kp sets saved: 45
1605510192 kp sets saved: 46
1605510192 kp sets saved: 47
1605510193 kp sets saved: 48
1605510193 kp sets saved: 49
1605510193 kp sets saved: 50
1605510193 kp sets saved: 51
1605510193 kp sets saved: 52
1605510193 kp sets saved: 53
1605510193 kp sets saved: 54
1605510193 kp sets saved: 55
1605510193 kp sets saved: 56
1605510193 kp sets saved: 57
1605510194 kp sets saved: 58
1605510194 kp sets saved: 59
1605510194 kp 

1605510219 kp sets saved: 98
1605510219 kp sets saved: 99
1605510219 kp sets saved: 100
1605510219 temp file copied to: /home/jetbot/Desktop/1605510219.csv
1605510220 temp file truncated
1605510220 kp sets saved: 1
1605510220 kp sets saved: 2
1605510220 kp sets saved: 3
1605510220 kp sets saved: 4
1605510220 kp sets saved: 5
1605510220 kp sets saved: 6
1605510220 kp sets saved: 7
1605510220 kp sets saved: 8
1605510221 kp sets saved: 9
1605510221 kp sets saved: 10
1605510221 kp sets saved: 11
1605510221 kp sets saved: 12
1605510221 kp sets saved: 13
1605510221 kp sets saved: 14
1605510221 kp sets saved: 15
1605510221 kp sets saved: 16
1605510221 kp sets saved: 17
1605510221 kp sets saved: 18
1605510222 kp sets saved: 19
1605510222 kp sets saved: 20
1605510222 kp sets saved: 21
1605510222 kp sets saved: 22
1605510222 kp sets saved: 23
1605510222 kp sets saved: 24
1605510222 kp sets saved: 25
1605510222 kp sets saved: 26
1605510222 kp sets saved: 27
1605510222 kp sets saved: 28
1605510223

...

1605510447 kp sets saved: 54
1605510447 kp sets saved: 55
1605510447 kp sets saved: 56
1605510447 kp sets saved: 57
1605510447 kp sets saved: 58
1605510447 kp sets saved: 59
1605510447 kp sets saved: 60
1605510447 kp sets saved: 61
1605510448 kp sets saved: 62
1605510448 kp sets saved: 63
1605510448 kp sets saved: 64
1605510448 kp sets saved: 65
1605510448 kp sets saved: 66
1605510448 kp sets saved: 67
1605510448 kp sets saved: 68
1605510448 kp sets saved: 69
1605510448 kp sets saved: 70
1605510448 kp sets saved: 71
1605510449 kp sets saved: 72
1605510449 kp sets saved: 73
1605510449 kp sets saved: 74
1605510449 kp sets saved: 75
1605510449 kp sets saved: 76
1605510449 kp sets saved: 77
1605510449 kp sets saved: 78
1605510449 kp sets saved: 79
1605510449 kp sets saved: 80
1605510449 kp sets saved: 81
1605510449 kp sets saved: 82
1605510450 kp sets saved: 83
1605510450 kp sets saved: 84
1605510450 kp sets saved: 85
1605510450 kp sets saved: 86
1605510450 kp sets saved: 87
1605510450 kp 

1605510474 kp sets saved: 26
1605510474 kp sets saved: 27
1605510474 kp sets saved: 28
1605510474 kp sets saved: 29
1605510475 kp sets saved: 30
1605510475 kp sets saved: 31
1605510475 kp sets saved: 32
1605510475 kp sets saved: 33
1605510475 kp sets saved: 34
1605510475 kp sets saved: 35
1605510475 kp sets saved: 36
1605510475 kp sets saved: 37
1605510475 kp sets saved: 38
1605510476 kp sets saved: 39
1605510476 kp sets saved: 40
1605510476 kp sets saved: 41
1605510476 kp sets saved: 42
1605510476 kp sets saved: 43
1605510476 kp sets saved: 44
1605510476 kp sets saved: 45
1605510476 kp sets saved: 46
1605510476 kp sets saved: 47
1605510476 kp sets saved: 48
1605510477 kp sets saved: 49
1605510477 kp sets saved: 50
1605510477 kp sets saved: 51
1605510477 kp sets saved: 52
1605510477 kp sets saved: 53
1605510477 kp sets saved: 54
1605510477 kp sets saved: 55
1605510477 kp sets saved: 56
1605510477 kp sets saved: 57
1605510477 kp sets saved: 58
1605510478 kp sets saved: 59
1605510478 kp 

1605510500 kp sets saved: 2
1605510500 kp sets saved: 3
1605510500 kp sets saved: 4
1605510500 kp sets saved: 5
1605510500 kp sets saved: 6
1605510500 kp sets saved: 7
1605510500 kp sets saved: 8
1605510500 kp sets saved: 9
1605510500 kp sets saved: 10
1605510500 kp sets saved: 11
1605510501 kp sets saved: 12
1605510501 kp sets saved: 13
1605510501 kp sets saved: 14
1605510501 kp sets saved: 15
1605510501 kp sets saved: 16
1605510501 kp sets saved: 17
1605510501 kp sets saved: 18
1605510501 kp sets saved: 19
1605510501 kp sets saved: 20
1605510501 kp sets saved: 21
1605510501 kp sets saved: 22
1605510502 kp sets saved: 23
1605510502 kp sets saved: 24
1605510502 kp sets saved: 25
1605510502 kp sets saved: 26
1605510502 kp sets saved: 27
1605510502 kp sets saved: 28
1605510502 kp sets saved: 29
1605510502 kp sets saved: 30
1605510502 kp sets saved: 31
1605510502 kp sets saved: 32
1605510502 kp sets saved: 33
1605510502 kp sets saved: 34
1605510503 kp sets saved: 35
1605510503 kp sets sav

1605510524 kp sets saved: 77
1605510524 kp sets saved: 78
1605510524 kp sets saved: 79
1605510524 kp sets saved: 80
1605510524 kp sets saved: 81
1605510524 kp sets saved: 82
1605510524 kp sets saved: 83
1605510524 kp sets saved: 84
1605510524 kp sets saved: 85
1605510524 kp sets saved: 86
1605510524 kp sets saved: 87
1605510525 kp sets saved: 88
1605510525 kp sets saved: 89
1605510525 kp sets saved: 90
1605510525 kp sets saved: 91
1605510525 kp sets saved: 92
1605510525 kp sets saved: 93
1605510525 kp sets saved: 94
1605510525 kp sets saved: 95
1605510525 kp sets saved: 96
1605510525 kp sets saved: 97
1605510525 kp sets saved: 98
1605510526 kp sets saved: 99
1605510526 kp sets saved: 100
1605510526 temp file copied to: /home/jetbot/Desktop/1605510526.csv
1605510526 temp file truncated
1605510526 kp sets saved: 1
1605510526 kp sets saved: 2
1605510526 kp sets saved: 3
1605510526 kp sets saved: 4
1605510526 kp sets saved: 5
1605510526 kp sets saved: 6
1605510526 kp sets saved: 7
16055105

1605510549 kp sets saved: 47
1605510549 kp sets saved: 48
1605510549 kp sets saved: 49
1605510549 kp sets saved: 50
1605510549 kp sets saved: 51
1605510549 kp sets saved: 52
1605510549 kp sets saved: 53
1605510550 kp sets saved: 54
1605510550 kp sets saved: 55
1605510550 kp sets saved: 56
1605510550 kp sets saved: 57
1605510550 kp sets saved: 58
1605510550 kp sets saved: 59
1605510550 kp sets saved: 60
1605510550 kp sets saved: 61
1605510550 kp sets saved: 62
1605510550 kp sets saved: 63
1605510550 kp sets saved: 64
1605510551 kp sets saved: 65
1605510551 kp sets saved: 66
1605510551 kp sets saved: 67
1605510551 kp sets saved: 68
1605510551 kp sets saved: 69
1605510551 kp sets saved: 70
1605510551 kp sets saved: 71
1605510551 kp sets saved: 72
1605510551 kp sets saved: 73
1605510551 kp sets saved: 74
1605510551 kp sets saved: 75
1605510552 kp sets saved: 76
1605510552 kp sets saved: 77
1605510552 kp sets saved: 78
1605510552 kp sets saved: 79
1605510552 kp sets saved: 80
1605510552 kp 

In [18]:
camera.unobserve_all()

1605510567 kp sets saved: 32
1605510567 kp sets saved: 33


In [19]:
f.close()

In [20]:
# Upload before exit
# This function call runs blocks because it doesn not run as a thread, so wait for it to finish
upload()
# Fan off
fan_control(0)

1605510571 Upload thread running...
1605510580 Uploads success, cleaning files...
1605510580 Files cleaned
1605510580 Upload thread finished
Changing speed of fan to: 0
Speed of fan changed
